In [1]:
# Handle different types of transformation
# different nb of pixels
# different nb of level of gray

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import tree
import os
import random

mpl.rc('image', cmap='gray_r')
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import load_model

## Useful methods

In [4]:
# Transformation matrices
def create_PPstar(transform_type, nbpix):
    if transform_type is "translation":
        # nb of pixels translated
        nbt = int(np.round(0.25*nbpix))

        # bottom left square block
        bl = np.eye(nbpix-nbt,nbpix-nbt)
        # upper right square block
        ur = np.eye(nbt,nbt)
        # upper left rect block
        ul = np.zeros((nbt,nbpix-nbt))
        # bottom right rect block
        br = np.zeros((nbpix-nbt,nbt))

        # concatenate the blocks to build the transformation matrix
        P = np.concatenate((np.concatenate((ul,ur), axis=1), np.concatenate((bl,br), axis=1)), axis=0)
        Pstar = np.linalg.inv(P)
    return (P,Pstar)

In [5]:
# plt.figure(1)
def remove_ticks():
    plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
    plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    left=False,
    labelleft=False,
    labelbottom=False) # labels along the bottom edge are off    
    return

def vis_matrices(Mr, Mm, Mst, MstM):
#     print(Mr.shape[0])
    if Mr.shape[0]==1:
        Mr = Mr.reshape(int(np.sqrt(Mr.shape[1])), int(np.sqrt(Mr.shape[1])))
        Mm = Mm.reshape(int(np.sqrt(Mm.shape[1])), int(np.sqrt(Mm.shape[1])))
        Mst = Mst.reshape(int(np.sqrt(Mst.shape[1])), int(np.sqrt(Mst.shape[1])))
        MstM = MstM.reshape(int(np.sqrt(Mst.shape[1])), int(np.sqrt(Mst.shape[1])))

    
    fig, (ax1,ax2) = plt.subplots(nrows=2,ncols=2) #, squeeze=True, sharey=True)
    fig.set_size_inches(8,8)

    plt.sca(ax1[0])
    plt.imshow(Mr)
    plt.title("Real image")
    remove_ticks()

    plt.sca(ax2[0])
    plt.imshow(Mm)
    plt.title("Distorted image")
    remove_ticks()

    plt.sca(ax1[1])
    plt.imshow(Mst)
    plt.title("Corrected input")
    remove_ticks()

    plt.sca(ax2[1])
    plt.imshow(MstM)
    plt.title("Corrected visualization")
    remove_ticks()

In [6]:
def create_training_data(nbpix,nb_lvl_gray=1):
    """
    Build the set of one pixel matrices and apply the transformations
    """
    # List of indexes where to put a black pixel
    idx = range(0,nbpix**2,1)

    # Initialize empty arrays 
    Mreal_tab = np.zeros((len(idx),nbpix*nbpix))
    Mmang_tab = np.zeros((len(idx),nbpix*nbpix))
    Mstar_tab = np.zeros((len(idx),nbpix*nbpix))
    for ii in idx:
        # Fill the indexed pixel with a one
        Mreal_tab[ii,ii] = 1
        # Use the transformation matrices to generate Mmang and Mstar
        Mmang_tab[ii,:] = (Mreal_tab[ii,:].reshape(nbpix,nbpix)@P).reshape(1,nbpix**2)
        Mstar_tab[ii,:] = (Mreal_tab[ii,:].reshape(nbpix,nbpix)@Pstar).reshape(1,nbpix**2)
    return (Mreal_tab, Mmang_tab, Mstar_tab)

In [7]:
# Function to add one on the designated index
def add_one(arr,index):
    arr[0,index] = 1
    return arr

# Function to split the matrix into matrices of one pixel
def split_matrix(mat):
    """ Split a matrix into matrices of one pixel"""
    # find the indexes of the ones values in the matrix
    idx_arr = np.where(mat.reshape(1,nbpix**2)[0]==1)[0]
    # transform from an array to a list
    idx_lst = [idx_arr.item(ii) for ii in range(0,len(idx_arr))] 
    temp = np.zeros((1,nbpix**2))
    mat_split = np.zeros((len(idx_lst), nbpix**2))
#     Mtest = np.zeros((nbpix,nbpix))
    for ii in range(0,len(idx_lst)):
        mat_split[ii,idx_lst[ii]] = 1    
#         Mtest = Mtest + Mreal_split[ii,:].reshape(nbpix,nbpix)
    return mat_split

In [8]:
# Prediction functions
def predict_mangler(mat):
    # split the input matrix into matrices of one pixel
    mat_split = split_matrix(mat)
    nb_dark_pxl = mat_split.shape[0]
    
    # initializations
    res = np.zeros((nb_dark_pxl, nbpix**2))
    Mres = np.zeros((nbpix,nbpix))
    # Loop on all the one pixels array
    for ii in range(0,nb_dark_pxl):
        # Apply the models to the one pixel matrices
        res[ii,:] = model_mangler.predict(np.expand_dims(mat_split[ii], axis=0))
        # Sum up the results to construct the matrices
        Mres = Mres + res[ii,:].reshape(nbpix,nbpix)
    return Mres

def predict_corrector(mat):
    # split the input matrix into matrices of one pixel
    mat_split = split_matrix(mat)
    nb_dark_pxl = mat_split.shape[0]
    
    # initializations
    res = np.zeros((nb_dark_pxl, nbpix**2))
    Mres = np.zeros((nbpix,nbpix))
    # Loop on all the one pixels array
    for ii in range(0,nb_dark_pxl):
        # Apply the models to the one pixel matrices
        res[ii,:] = model_corrector.predict(np.expand_dims(mat_split[ii], axis=0))
        # Sum up the results to construct the matrices
        Mres = Mres + res[ii,:].reshape(nbpix,nbpix)
    return Mres

In [37]:
def create_door_img(nbpix):
    """ Create an nbpix by nbpix image of a door"""
    nbborder = int(np.round(nbpix*0.3))
    u = np.ones((nbborder,nbpix))
    l = np.ones((nbpix-nbborder,nbborder))
    r = l
    door = np.zeros((nbpix-nbborder, nbpix-2*nbborder))

    return np.concatenate((u,np.concatenate((l,door,r), axis=1)), axis=0)

 ## MAIN

In [56]:
# Parameters
nbpix = 9 # nb of pixel to consider (nbpix x nbpix)
transform_type = "translation" # type of transformation
nb_lvl_gray = 1 # nb of level of gray

In [57]:
# create the transformation matrices
(P,Pstar) = create_PPstar("translation",nbpix)

# Create the training data
(Mreal_tab, Mmang_tab, Mstar_tab) = create_training_data(nbpix,1)
idx = range(0,nbpix**2,1)

# the training input are the matrices of the real images with one pixel
X_train = Mreal_tab

# the training output will be the category corresponding to the position of the pixel in the mangled image
y_train = [np.where(Mmang_tab[ii,:]==1)[0].item(0) for ii in idx]
y_train = to_categorical(y_train, nbpix**2)

# or the position of the pixel in the corrected image
y_train_cor = [np.where(Mstar_tab[ii,:]==1)[0].item(0) for ii in idx]
y_train_cor = to_categorical(y_train_cor, nbpix**2)

# creating the models
model_mangler = Sequential()
model_mangler.add(Dense(4*nbpix**2, activation='relu', input_dim=nbpix**2))
model_mangler.add(Dense(4*nbpix**2, activation='relu'))
model_mangler.add(Dense(nbpix**2, activation='softmax'))
# Inversion strategy doesnt work very well...
# we are going to try using another model, trained on the corrected input.
model_corrector = Sequential()
model_corrector.add(Dense(4*nbpix**2, activation='relu', input_dim=nbpix**2))
model_corrector.add(Dense(4*nbpix**2, activation='relu'))
model_corrector.add(Dense(nbpix**2, activation='softmax'))

# Compile the models
model_mangler.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])
model_corrector.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

# train the model "mangler"
model_mangler.fit( X_train,
    y_train,
    epochs=5*nbpix**2,
    shuffle=True,
    verbose=1
)

# train the corrected input model
model_corrector.fit( X_train,
    y_train_cor,
    epochs=5*nbpix**2,
    shuffle=True,
    verbose=1
)    

# model_mangler.summary()
# model_corrector.summary()

# Save the models
model_mangler.save(f"{transform_type}_mangler_{nbpix}x{nbpix}.h5")
model_corrector.save(f"{transform_type}_corrector_{nbpix}x{nbpix}.h5")


Epoch 1/405
81/81 [==============================] - 1s 15ms/step - loss: 4.4078 - acc: 0.0000e+00
Epoch 2/405
81/81 [==============================] - 0s 420us/step - loss: 4.3388 - acc: 0.1235
Epoch 3/405
81/81 [==============================] - 0s 482us/step - loss: 4.2833 - acc: 0.4815
Epoch 4/405
81/81 [==============================] - 0s 309us/step - loss: 4.2285 - acc: 0.7037
Epoch 5/405
81/81 [==============================] - 0s 321us/step - loss: 4.1695 - acc: 0.8642
Epoch 6/405
81/81 [==============================] - 0s 247us/step - loss: 4.1059 - acc: 0.9259
Epoch 7/405
81/81 [==============================] - 0s 333us/step - loss: 4.0349 - acc: 0.9630
Epoch 8/405
81/81 [==============================] - 0s 284us/step - loss: 3.9574 - acc: 1.0000
Epoch 9/405
81/81 [==============================] - 0s 296us/step - loss: 3.8702 - acc: 1.0000
Epoch 10/405
81/81 [==============================] - 0s 370us/step - loss: 3.7765 - acc: 1.0000
Epoch 11/405
81/81 [================

81/81 [==============================] - 0s 198us/step - loss: 0.0081 - acc: 1.0000
Epoch 86/405
81/81 [==============================] - 0s 185us/step - loss: 0.0079 - acc: 1.0000
Epoch 87/405
81/81 [==============================] - 0s 185us/step - loss: 0.0077 - acc: 1.0000
Epoch 88/405
81/81 [==============================] - 0s 284us/step - loss: 0.0076 - acc: 1.0000
Epoch 89/405
81/81 [==============================] - 0s 210us/step - loss: 0.0074 - acc: 1.0000
Epoch 90/405
81/81 [==============================] - 0s 222us/step - loss: 0.0072 - acc: 1.0000
Epoch 91/405
81/81 [==============================] - 0s 235us/step - loss: 0.0071 - acc: 1.0000
Epoch 92/405
81/81 [==============================] - 0s 259us/step - loss: 0.0069 - acc: 1.0000
Epoch 93/405
81/81 [==============================] - 0s 321us/step - loss: 0.0068 - acc: 1.0000
Epoch 94/405
81/81 [==============================] - 0s 370us/step - loss: 0.0067 - acc: 1.0000
Epoch 95/405
81/81 [=======================

81/81 [==============================] - 0s 272us/step - loss: 0.0023 - acc: 1.0000
Epoch 169/405
81/81 [==============================] - 0s 296us/step - loss: 0.0022 - acc: 1.0000
Epoch 170/405
81/81 [==============================] - 0s 235us/step - loss: 0.0022 - acc: 1.0000
Epoch 171/405
81/81 [==============================] - 0s 247us/step - loss: 0.0022 - acc: 1.0000
Epoch 172/405
81/81 [==============================] - 0s 407us/step - loss: 0.0022 - acc: 1.0000
Epoch 173/405
81/81 [==============================] - 0s 333us/step - loss: 0.0022 - acc: 1.0000
Epoch 174/405
81/81 [==============================] - 0s 284us/step - loss: 0.0021 - acc: 1.0000
Epoch 175/405
81/81 [==============================] - 0s 383us/step - loss: 0.0021 - acc: 1.0000
Epoch 176/405
81/81 [==============================] - 0s 321us/step - loss: 0.0021 - acc: 1.0000
Epoch 177/405
81/81 [==============================] - 0s 296us/step - loss: 0.0021 - acc: 1.0000
Epoch 178/405
81/81 [=============

81/81 [==============================] - 0s 284us/step - loss: 0.0011 - acc: 1.0000
Epoch 252/405
81/81 [==============================] - 0s 284us/step - loss: 0.0011 - acc: 1.0000
Epoch 253/405
81/81 [==============================] - 0s 296us/step - loss: 0.0011 - acc: 1.0000
Epoch 254/405
81/81 [==============================] - 0s 383us/step - loss: 0.0011 - acc: 1.0000
Epoch 255/405
81/81 [==============================] - 0s 358us/step - loss: 0.0010 - acc: 1.0000
Epoch 256/405
81/81 [==============================] - 0s 309us/step - loss: 0.0010 - acc: 1.0000
Epoch 257/405
81/81 [==============================] - 0s 383us/step - loss: 0.0010 - acc: 1.0000
Epoch 258/405
81/81 [==============================] - 0s 395us/step - loss: 0.0010 - acc: 1.0000
Epoch 259/405
81/81 [==============================] - 0s 346us/step - loss: 0.0010 - acc: 1.0000
Epoch 260/405
81/81 [==============================] - 0s 358us/step - loss: 0.0010 - acc: 1.0000
Epoch 261/405
81/81 [=============

81/81 [==============================] - 0s 259us/step - loss: 6.4028e-04 - acc: 1.0000
Epoch 332/405
81/81 [==============================] - 0s 247us/step - loss: 6.3654e-04 - acc: 1.0000
Epoch 333/405
81/81 [==============================] - 0s 247us/step - loss: 6.3291e-04 - acc: 1.0000
Epoch 334/405
81/81 [==============================] - 0s 346us/step - loss: 6.2925e-04 - acc: 1.0000
Epoch 335/405
81/81 [==============================] - 0s 358us/step - loss: 6.2565e-04 - acc: 1.0000
Epoch 336/405
81/81 [==============================] - 0s 358us/step - loss: 6.2198e-04 - acc: 1.0000
Epoch 337/405
81/81 [==============================] - 0s 383us/step - loss: 6.1850e-04 - acc: 1.0000
Epoch 338/405
81/81 [==============================] - 0s 407us/step - loss: 6.1504e-04 - acc: 1.0000
Epoch 339/405
81/81 [==============================] - 0s 370us/step - loss: 6.1156e-04 - acc: 1.0000
Epoch 340/405
81/81 [==============================] - 0s 358us/step - loss: 6.0809e-04 - acc: 1

81/81 [==============================] - 0s 198us/step - loss: 4.0982 - acc: 0.9753
Epoch 7/405
81/81 [==============================] - 0s 259us/step - loss: 4.0255 - acc: 1.0000
Epoch 8/405
81/81 [==============================] - 0s 198us/step - loss: 3.9465 - acc: 1.0000
Epoch 9/405
81/81 [==============================] - 0s 309us/step - loss: 3.8578 - acc: 1.0000
Epoch 10/405
81/81 [==============================] - 0s 383us/step - loss: 3.7603 - acc: 1.0000
Epoch 11/405
81/81 [==============================] - 0s 321us/step - loss: 3.6526 - acc: 1.0000
Epoch 12/405
81/81 [==============================] - 0s 346us/step - loss: 3.5328 - acc: 1.0000
Epoch 13/405
81/81 [==============================] - 0s 296us/step - loss: 3.4021 - acc: 1.0000
Epoch 14/405
81/81 [==============================] - 0s 358us/step - loss: 3.2572 - acc: 1.0000
Epoch 15/405
81/81 [==============================] - 0s 309us/step - loss: 3.0994 - acc: 1.0000
Epoch 16/405
81/81 [==========================

81/81 [==============================] - 0s 247us/step - loss: 0.0072 - acc: 1.0000
Epoch 91/405
81/81 [==============================] - 0s 247us/step - loss: 0.0070 - acc: 1.0000
Epoch 92/405
81/81 [==============================] - 0s 222us/step - loss: 0.0069 - acc: 1.0000
Epoch 93/405
81/81 [==============================] - 0s 247us/step - loss: 0.0068 - acc: 1.0000
Epoch 94/405
81/81 [==============================] - 0s 296us/step - loss: 0.0066 - acc: 1.0000
Epoch 95/405
81/81 [==============================] - 0s 383us/step - loss: 0.0065 - acc: 1.0000
Epoch 96/405
81/81 [==============================] - 0s 358us/step - loss: 0.0064 - acc: 1.0000
Epoch 97/405
81/81 [==============================] - 0s 370us/step - loss: 0.0062 - acc: 1.0000
Epoch 98/405
81/81 [==============================] - 0s 296us/step - loss: 0.0061 - acc: 1.0000
Epoch 99/405
81/81 [==============================] - 0s 333us/step - loss: 0.0060 - acc: 1.0000
Epoch 100/405
81/81 [======================

81/81 [==============================] - 0s 284us/step - loss: 0.0021 - acc: 1.0000
Epoch 174/405
81/81 [==============================] - 0s 222us/step - loss: 0.0021 - acc: 1.0000
Epoch 175/405
81/81 [==============================] - 0s 259us/step - loss: 0.0021 - acc: 1.0000
Epoch 176/405
81/81 [==============================] - 0s 321us/step - loss: 0.0021 - acc: 1.0000
Epoch 177/405
81/81 [==============================] - 0s 383us/step - loss: 0.0021 - acc: 1.0000
Epoch 178/405
81/81 [==============================] - 0s 358us/step - loss: 0.0020 - acc: 1.0000
Epoch 179/405
81/81 [==============================] - 0s 296us/step - loss: 0.0020 - acc: 1.0000
Epoch 180/405
81/81 [==============================] - 0s 284us/step - loss: 0.0020 - acc: 1.0000
Epoch 181/405
81/81 [==============================] - 0s 321us/step - loss: 0.0020 - acc: 1.0000
Epoch 182/405
81/81 [==============================] - 0s 272us/step - loss: 0.0020 - acc: 1.0000
Epoch 183/405
81/81 [=============

Epoch 256/405
81/81 [==============================] - 0s 247us/step - loss: 0.0010 - acc: 1.0000
Epoch 257/405
81/81 [==============================] - 0s 235us/step - loss: 0.0010 - acc: 1.0000
Epoch 258/405
81/81 [==============================] - 0s 235us/step - loss: 0.0010 - acc: 1.0000
Epoch 259/405
81/81 [==============================] - 0s 383us/step - loss: 0.0010 - acc: 1.0000
Epoch 260/405
81/81 [==============================] - 0s 321us/step - loss: 0.0010 - acc: 1.0000
Epoch 261/405
81/81 [==============================] - 0s 358us/step - loss: 0.0010 - acc: 1.0000
Epoch 262/405
81/81 [==============================] - 0s 370us/step - loss: 9.9645e-04 - acc: 1.0000
Epoch 263/405
81/81 [==============================] - 0s 407us/step - loss: 9.8933e-04 - acc: 1.0000
Epoch 264/405
81/81 [==============================] - 0s 407us/step - loss: 9.8240e-04 - acc: 1.0000
Epoch 265/405
81/81 [==============================] - 0s 395us/step - loss: 9.7550e-04 - acc: 1.0000
Epoc

81/81 [==============================] - 0s 259us/step - loss: 6.2376e-04 - acc: 1.0000
Epoch 337/405
81/81 [==============================] - 0s 247us/step - loss: 6.2027e-04 - acc: 1.0000
Epoch 338/405
81/81 [==============================] - 0s 272us/step - loss: 6.1672e-04 - acc: 1.0000
Epoch 339/405
81/81 [==============================] - 0s 272us/step - loss: 6.1332e-04 - acc: 1.0000
Epoch 340/405
81/81 [==============================] - 0s 321us/step - loss: 6.0983e-04 - acc: 1.0000
Epoch 341/405
81/81 [==============================] - 0s 321us/step - loss: 6.0645e-04 - acc: 1.0000
Epoch 342/405
81/81 [==============================] - 0s 284us/step - loss: 6.0304e-04 - acc: 1.0000
Epoch 343/405
81/81 [==============================] - 0s 358us/step - loss: 5.9970e-04 - acc: 1.0000
Epoch 344/405
81/81 [==============================] - 0s 383us/step - loss: 5.9637e-04 - acc: 1.0000
Epoch 345/405
81/81 [==============================] - 0s 358us/step - loss: 5.9303e-04 - acc: 1

In [55]:
# Apply the models to an existing image

# Load the saved models
model_mangler = load_model(f"{transform_type}_mangler_{nbpix}x{nbpix}.h5")
model_corrector = load_model(f"{transform_type}_corrector_{nbpix}x{nbpix}.h5")

Mreal_split = split_matrix(Mreal)

Mreal = create_door_img(nbpix) 

ValueError: cannot reshape array of size 36 into shape (1,25)